# Question 2022-02-03: Plot NP vs T; Phase Composition vs T
Seem to get strange behaviour with multiple versions of the same phase appearing, not summing to unity.

In [ ]:
from pycalphad import Database, equilibrium, variables as v
import numpy as np

num_points = 2 # Number of concentration values for each element
T_start = 600.0 # degrees C
T_end = 1300.0 # degrees C
T_inc = 100.0 # degrees C
Pressure = 101325.0 # Pa

my_els = ['NI', 'AL', 'CO', 'CR', 'MO', 'TI', 'VA']
concentrations = [0.064, 0.18, 0.16, 0.030, 0.043] # bulk composition
concentrations_gamma = [0.0587, 0.1955, 0.1828, 0.03, 0.0421]
concentrations_gamma_prime = [0.123, 0.06, 0.025, 0.0006, 0.087] # gamma prime composition

my_phases_alni = ['FCC_A1']

dbf = Database('mc_ni_v2.034.tdb')

concentrations_gamma = np.array(concentrations_gamma)
concentrations_gamma_prime = np.array(concentrations_gamma_prime)

concentrations_min = []
concentrations_max = []

for counter in range(len(concentrations_gamma)):
    if concentrations_gamma[counter] < concentrations_gamma_prime[counter]:
        concentrations_min += [concentrations_gamma[counter]]
        concentrations_max += [concentrations_gamma_prime[counter]]
    elif concentrations_gamma[counter] > concentrations_gamma_prime[counter]:
        concentrations_max += [concentrations_gamma[counter]]
        concentrations_min += [concentrations_gamma_prime[counter]]


my_vars = {v.X('AL'):(np.linspace(concentrations_min[0], concentrations_max[0], num_points)),
           v.X('CO'):(np.linspace(concentrations_min[1], concentrations_max[1], num_points)),
           v.X('CR'):(np.linspace(concentrations_min[2], concentrations_max[2], num_points)),
           v.X('MO'):(np.linspace(concentrations_min[3], concentrations_max[3], num_points)),
           v.X('TI'):(np.linspace(concentrations_min[4], concentrations_max[4], num_points)),
           v.T:(T_start+273.15, T_end+273.15, T_inc),
           v.P:Pressure}

eq = equilibrium(dbf, my_els, my_phases_alni, my_vars)

## NP vs. T

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pycalphad.plot.utils import phase_legend
phases = sorted(set(eq.Phase.values.flatten()) - {''})
phase_handles, phasemap = phase_legend(phases)

plt.gca().set_title('Phase fractions vs T')
plt.gca().set_xlabel('Temperature (Celsius)')
plt.gca().set_ylabel('Phase Fraction')
plt.gca().set_ylim((0,1.1))
plt.gca().set_xlim((T_start, T_end))

for name in phases:
    plt.scatter(eq.T.broadcast_like(eq.NP)-273.15, eq.NP.where(eq.Phase == my_phases_alni[0]), color=phasemap[name])
plt.gca().legend(phase_handles, phases, loc='lower right')